In [1]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==1

In [3]:
import io, copy, json
from datasets import load_dataset
import ipywidgets as w
from IPython.display import display, clear_output

# 🔧 1)  EDIT THIS ---------------------------------
REPO_ID = "johnowhitaker/specid_eval_jw"  # HF repo with the image dataset
# -----------------------------------------------

ds = load_dataset(REPO_ID, split="train")  # expects keys: image  options  answer

# Global bookkeeping
i        = 0            # current index
n        = len(ds)      # total samples
correct  = 0            # running correct count
guesses  = []           # store what you clicked

# Widget placeholders (updated each round)
lbl_progress = w.HTML()
lbl_score    = w.HTML()

def ask(idx:int):
    """Render one question and attach click‑handlers that advance to the next."""
    clear_output(wait=True)

    ex         = ds[idx]
    options    = copy.deepcopy(ex["options"])          # don't mutate HF data
    while len(options) < 5:                            # pad to 5 buttons
        options.append("NA")

    # Image ➜ widget
    buf = io.BytesIO()
    ex["image"].save(buf, format="PNG")
    img = w.Image(value=buf.getvalue(), format="png", layout=w.Layout(width="320px"))

    # Option buttons
    btns = [w.Button(description=o, layout=w.Layout(width="auto"))
            for o in options]

    def on_click(b):
        nonlocal idx
        global i, correct

        guess = b.description
        guesses.append(guess)

        # update running accuracy
        if guess == options[ex["answer"]]:
            correct += 1

        i += 1
        if i < n:                                     # go to next image
            ask(i)
        else:                                         # finished the set
            clear_output()
            final_acc = correct / n * 100
            print(f"🎉  Done! You labeled {n} images with {final_acc:.1f}% accuracy.")
            # Save guesses to JSON for later use
            with open("human_baseline.json", "w") as f:
                json.dump({"guesses": guesses, "accuracy": final_acc}, f)
            print("Saved guesses ➜ human_baseline.json")

    # Attach handler *after* definition
    for b in btns:
        b.on_click(on_click)

    # Progress & score labels
    lbl_progress.value = f"<b>Progress:</b> {idx+1}/{n}"
    lbl_score.value    = ("" if idx == 0 else
                          f"<b>Current accuracy:</b> {correct/idx*100:.1f}%")

    # Render the UI
    ui = w.VBox([
        lbl_progress,
        img,
        w.HBox(btns),
        lbl_score,
    ])
    display(ui)

ask(0)        # 🚀 kick off the first question

🎉  Done! You labeled 100 images with 71.0% accuracy.
Saved guesses ➜ human_baseline.json
